In [2]:
# Use Jupyter Black for cell formatting
import jupyter_black

jupyter_black.load()

---

# Imports and Data Load

In [21]:
# Imports
import numpy as np
import polars as pl
import polars.selectors as cs
import plotly.express as px
import plotly.figure_factory as ff
import scipy.stats as stats

In [4]:
# Data Load
df = pl.read_csv("./data/rotten_tomatoes_movies.csv")

---

# Initial Data Exploration

In [ ]:
# Data Shape
df.shape

In [6]:
# Convert strings to dates
df = df.with_columns(
    pl.col("in_theaters_date").str.to_date(),
    pl.col("on_streaming_date").str.to_date(),
)

In [ ]:
# DTypes
df.dtypes

In [ ]:
# Missing data
df.null_count()

In [ ]:
# Column name to drop rows when null
column_list = [
    "genre",
    "directors",
    "in_theaters_date",
    "on_streaming_date",
    "audience_rating",
    "audience_count",
]

df = df.filter(~pl.any_horizontal(pl.col(column_list).is_null()))

df.shape

In [ ]:
# Missing data
df.null_count()

In [ ]:
# Describe the Date and Numeric Columns
df.select(cs.by_dtype(pl.Date, pl.Int64)).describe()

---

# Exploratory Data Analysis

In [ ]:
# Barcharts of the categorical data
fig_01 = px.histogram(
    df, x="rating", title="Rating Histogram (before cleaning)", width=600, height=400
)
fig_01.show()

In [13]:
# Clean up Rating data
df = df.with_columns(
    pl.col("rating").str.replace_many(
        ["PG-13)", "R)"],
        ["PG-13", "R"],
    )
)

In [ ]:
# Barcharts of the categorical data
fig_01_new = px.histogram(
    df, x="rating", title="Rating Histogram (after cleaning)", width=600, height=400
)
fig_01_new.show()

In [ ]:
fig_02 = px.histogram(
    df,
    x="tomatometer_status",
    title="Tomato Meter Status Histogram",
    width=600,
    height=400,
)
fig_02.show()

---

## Distibution plots of the Ratings

Look at both the Tamoto Meter and Audience Ratings

In [ ]:
# Group data together
hist_data = [
    df["tomatometer_rating"].to_numpy(),
    df["audience_rating"].to_numpy(),
]

group_labels = [
    "Tomatometer Rating",
    "Audience Rating",
]

# Create distplot with custom bin_size
fig_03 = ff.create_distplot(hist_data, group_labels, bin_size=5, show_rug=False)
fig_03.update_layout(
    width=600,
    height=400,
    title="Distribution Plots for Audience Rating and Tomatometer Rating",
    xaxis_title="Rating",
    yaxis_title="Distribution",
)
fig_03.show()

In [ ]:
print(f"Tomato Meter Rating Skew: {df["tomatometer_rating"].skew()}")
print(f"Audience Rating Skew: {df["audience_rating"].skew()}")

In [ ]:
# The Mean Values
round(df["tomatometer_rating"].mean(), 2), round(df["audience_rating"].mean(), 2)

In [ ]:
# Set Alpha
alpha = 0.05

# Calculate and Print
t_value1, p_value1 = stats.ttest_ind(df["tomatometer_rating"], df["audience_rating"])
print(f"t_value1: {t_value1}\np_value1: {p_value1}")

Next, although optional, it is useful to print 'if/else' statements to make our conclusions about the the hypothesis.


In [ ]:
if p_value1 < alpha:
    print("Conclusion: since p_value {} is less than alpha {} ".format(p_value1, alpha))
    print(
        "Reject the null hypothesis that there is no difference between bmi of females and bmi of males."
    )

else:
    print(
        "Conclusion: since p_value {} is greater than alpha {} ".format(p_value1, alpha)
    )
    print(
        "Fail to reject the null hypothesis that there is a difference between bmi of females and bmi of males."
    )

---

### Theatre Date and Streaming Date Analysis

In [ ]:
# Histogram of the Streaming date
# Columns for the Streaming Year and Month
df = df.with_columns(
    pl.col("in_theaters_date").dt.year().alias("theatre_year"),
    pl.col("in_theaters_date").dt.month().alias("theatre_month"),
    pl.col("on_streaming_date").dt.year().alias("streaming_year"),
    pl.col("on_streaming_date").dt.month().alias("streaming_month"),
)

fig_04 = px.box(
    df,
    x="streaming_year",
    title="Streaming Date Boxplot",
    width=600,
    height=400,
    points="all",
)
fig_04.show()

In [ ]:
# Movies Streamed before 1990
df.filter(pl.col("streaming_year").lt(1990))

In [20]:
# Fix theatre and stream dates for The 39 Steps
df = df.with_columns(
    pl.when(pl.col("movie_title").eq("The 39 Steps"))
    .then(pl.date(1935, 6, 6))
    .otherwise(pl.col("in_theaters_date"))
    .alias("in_theaters_date")
).with_columns(
    pl.when(pl.col("movie_title").eq("The 39 Steps"))
    .then(pl.date(2017, 1, 12))
    .otherwise(pl.col("on_streaming_date"))
    .alias("on_streaming_date")
)

In [ ]:
# Violin of the Streaming date
fig_04 = px.violin(
    df,
    x="streaming_year",
    title="Streaming Date Violin Plot",
    width=600,
    height=400,
    points="all",
)
fig_04.show()

In [ ]:
# The number of days, months, years between the
# Theatre date and the streaming date
df = df.with_columns(
    (pl.col("on_streaming_date") - pl.col("in_theaters_date")).alias("days_between")
)

# How many films have a streaming date before an in theatre date
df.filter(pl.col("days_between").lt(0)).shape

---

## Movie Runtime

In [ ]:
# Boxplot of the Runtime
fig_05 = px.box(
    df.filter(pl.col("runtime_in_minutes").is_not_null()),
    x="runtime_in_minutes",
    title="Runtime in Minutes Boxplot",
    width=600,
    height=400,
    points="all",
)
fig_05.show()

In [ ]:
# Runtimes longer than 400 minutes
df.filter(pl.col("runtime_in_minutes").gt(400))

In [25]:
# Fill nulll runtime values with the median
df = df.with_columns(
    pl.col("runtime_in_minutes").fill_null(pl.col("runtime_in_minutes").median())
)

mean_runtime = int(df["runtime_in_minutes"].mean())

# Fix movies with excessive runtimes
# This can be done using the actual values
df = df.with_columns(
    pl.col("runtime_in_minutes").replace([496, 2000, 480], [80, 93, mean_runtime])
)

In [ ]:
# Violin plot of the runtime
fig_06 = px.violin(
    df,
    x="runtime_in_minutes",
    title="Runtime in Minutes Violin Plot",
    width=600,
    height=400,
    points="all",
)
fig_06.show()

---

## Pair Plots and Correlation

In [ ]:
# Average Tomatometer and Audience Ratings Over Time
fig_07 = px.line(
    df.group_by("theatre_year")
    .agg(
        pl.col("tomatometer_rating").mean().alias("average_tomatometer"),
        pl.col("audience_rating").mean().alias("average_audience"),
    )
    .sort("theatre_year"),
    x="theatre_year",
    y=["average_tomatometer", "average_audience"],
    width=600,
    height=400,
    title="Average Tomatometer and Audience Ratings Over Time",
)
fig_07.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99))
fig_07.show()

In [ ]:
# Audience vs. Tomatometer Rating Comparison
fig_08 = px.scatter(
    df,
    x="tomatometer_rating",
    y="audience_rating",
    trendline="ols",
    trendline_color_override="red",
    width=600,
    height=400,
    title="Audience Rating vs. Tomatometer Rating<br><sup>Including Trendline</sup>",
)
fig_08.update_traces(marker_size=10, opacity=0.25)
fig_08.data[1].update(line_color="red", line_width=4)
fig_08.update_layout(
    xaxis_title="Tomatometer Rating",
    yaxis_title="Audience Rating",
)

fig_08.show()

In [ ]:
df.select(pl.col("tomatometer_rating"), pl.col("audience_rating")).corr()

In [ ]:
df_corr = df.select(pl.col(pl.Float64, pl.Int64)).corr().with_columns(pl.all().round(3))

df_corr = df_corr.rename(
    {
        "runtime_in_minutes": "runtime",
        "tomatometer_rating": "tm_rating",
        "tomatometer_count": "tm_count",
        "audience_rating": "aud_rating",
        "audience_count": "aud_count",
    }
)

fig_09 = px.imshow(
    df_corr,
    text_auto=True,
    color_continuous_scale="RdBu_r",
    width=600,
    height=400,
    title="Numeric Data Correlation Heatmap",
)

fig_09.update_layout(
    yaxis=dict(
        tickmode="array",
        tickvals=[0, 1, 2, 3, 4],
        ticktext=df_corr.columns,
    )
)

fig_09.show()

---

## Feature Engineering

In [ ]:
# https://stackoverflow.com/questions/74432537/cast-a-list-column-into-dummy-columns-in-python-polars
df_genre = (
    (
        df.with_columns(
            pl.int_range(pl.len()).alias("row_index"),
            pl.col("genre").str.split(", "),
            pl.lit(1).alias("__one__"),
        )
        .explode("genre")
        .pivot(
            on="genre", index="row_index", values="__one__", aggregate_function="first"
        )
        .fill_null(0)
    )
    .drop("row_index")
    .sum()
    .transpose(include_header=True)
    .rename({"column": "genre", "column_0": "count_of_movies"})
    .sort("count_of_movies")
)

In [ ]:
fig_10 = px.bar(
    df_genre,
    x="count_of_movies",
    y="genre",
    title="Most Popular Genres",
    width=600,
    height=500,
    orientation="h",
)
fig_10.update_layout(
    xaxis_title="Count of Movies",
    yaxis_title="Genre",
)

fig_10.show()

In [162]:
df_director = (
    (
        df.with_columns(
            pl.int_range(pl.len()).alias("row_index"),
            pl.col("directors").str.split(", "),
            pl.lit(1).alias("__one__"),
        )
        .explode("directors")
        .pivot(
            on="directors",
            index="row_index",
            values="__one__",
            aggregate_function="first",
        )
        .fill_null(0)
    )
    .drop("row_index")
    .sum()
    .transpose(include_header=True)
    .rename({"column": "directors", "column_0": "count_of_movies"})
    .sort("count_of_movies")
)

In [ ]:
(
    df_director.shape,
    df_director.select("count_of_movies").max(),
    df_director.select("count_of_movies").mean(),
    df_director.select("count_of_movies").median(),
)

In [165]:
df_writer = (
    (
        df.with_columns(
            pl.int_range(pl.len()).alias("row_index"),
            pl.col("writers").str.split(", "),
            pl.lit(1).alias("__one__"),
        )
        .explode("writers")
        .pivot(
            on="writers",
            index="row_index",
            values="__one__",
            aggregate_function="first",
        )
        .fill_null(0)
    )
    .drop("row_index")
    .sum()
    .transpose(include_header=True)
    .rename({"column": "writers", "column_0": "count_of_movies"})
    .sort("count_of_movies")
)

In [ ]:
(
    df_writer.shape,
    df_writer.select("count_of_movies").max(),
    df_writer.select("count_of_movies").mean(),
    df_writer.select("count_of_movies").median(),
)

In [168]:
df_cast = (
    (
        df.with_columns(
            pl.int_range(pl.len()).alias("row_index"),
            pl.col("cast").str.split(", "),
            pl.lit(1).alias("__one__"),
        )
        .explode("cast")
        .pivot(
            on="cast",
            index="row_index",
            values="__one__",
            aggregate_function="first",
        )
        .fill_null(0)
    )
    .drop("row_index")
    .sum()
    .transpose(include_header=True)
    .rename({"column": "cast", "column_0": "count_of_movies"})
    .sort("count_of_movies")
)

In [ ]:
(
    df_cast.shape,
    df_cast.select("count_of_movies").max(),
    df_cast.select("count_of_movies").mean(),
    df_cast.select("count_of_movies").median(),
)

In [ ]:
df.head()